In [83]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [84]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [85]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [86]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.createOrReplaceTempView("home_sales")
# Show DataFrame
df.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [87]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, year, round

df = df.withColumn("price", col("price").cast("double"))

# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
fourbed_avg_price_by_year = df.filter(col("bedrooms") == 4).groupBy(year("date").alias("year")).avg("price")

# Round the average price to two decimal places
fourbed_avg_price_by_year = fourbed_avg_price_by_year.withColumn("avg(price)", round(col("avg(price)"), 2))

# Show the result
fourbed_avg_price_by_year.show()



+----+----------+
|year|avg(price)|
+----+----------+
|2022| 296363.88|
|2019|  300263.7|
|2020| 298353.78|
|2021| 301819.44|
+----+----------+



In [88]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: double (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [89]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
threebedbath_avg_price_by_year = df.filter((col("bedrooms") == 3) & (col("bathrooms") == 3)).groupBy(year("date").alias("year")).avg("price")

# Round the average price to two decimal places
threebedbath_avg_price_by_year = threebedbath_avg_price_by_year.withColumn("avg(price)", round(col("avg(price)"), 2))

threebedbath_avg_price_by_year.show()


+----+----------+
|year|avg(price)|
+----+----------+
|2022| 292725.69|
|2019| 287287.82|
|2020| 294204.16|
|2021| 294211.46|
+----+----------+



In [90]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
df = df.withColumn("sqft_living", col("sqft_living").cast("double"))
specified_avg_price_by_year = df.filter((col("bedrooms") == 3) & (col("bathrooms") == 3) & (col("floors") == 2) & (col("sqft_living") >= 2000)).groupBy(year("date").alias("year")).avg("price")
# Round the average price to two decimal places
specified_avg_price_by_year = specified_avg_price_by_year.withColumn("avg(price)", round(col("avg(price)"), 2))
specified_avg_price_by_year.show()

+----+----------+
|year|avg(price)|
+----+----------+
|2022| 290242.99|
|2019| 289859.14|
|2020| 292289.09|
|2021| 296330.96|
+----+----------+



In [91]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.
# Convert the "view" column to DoubleType
df = df.withColumn("view", col("view").cast("double"))

# Calculate the average view rating for homes with a price greater than or equal to $350,000
view_avg = df.filter(col("price") >= 350000).agg({"view": "avg"})

# Round the average view rating to two decimal places
view_avg = view_avg.withColumnRenamed("avg(view)", "avg_view").withColumn("avg_view", round(col("avg_view"), 2))

# Show the result
view_avg.show()

# Measure the run time of the query
start_time = time.time()
view_avg.collect()  # Execute the query and collect the result
print("--- %s seconds ---" % (time.time() - start_time))

+--------+
|avg_view|
+--------+
|   32.26|
+--------+

--- 0.7018201351165771 seconds ---


In [92]:
# 7. Cache the the temporary table home_sales.
spark.table("home_sales").cache()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [93]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
# Query 1: Using the cached data
start_time = time.time()

# Run the query on the cached DataFrame
cached_query_result = spark.sql("SELECT * FROM home_sales WHERE view >= 350000")

# Calculate the runtime for the cached query
cached_runtime = time.time() - start_time

print("--- Cached Runtime: %s seconds ---" % cached_runtime)

# Uncache the DataFrame
spark.table("home_sales").unpersist()

# Query 2: Using the uncached data
start_time = time.time()

# Run the same query on the original DataFrame without caching
uncached_query_result = df.filter(col("view") >= 350000)

# Calculate the runtime for the uncached query
uncached_runtime = time.time() - start_time

print("--- Uncached Runtime: %s seconds ---" % uncached_runtime)




--- Cached Runtime: 0.02991485595703125 seconds ---
--- Uncached Runtime: 0.03620481491088867 seconds ---


In [104]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [109]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("/content/gdrive/My Drive/Home_Sales_Colab1/")



In [110]:
# 11. Read the parquet formatted data.
df_parquet = spark.read.parquet("/content/gdrive/My Drive/Home_Sales_Colab1/")

In [111]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView("parquet_data")

In [112]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
# Run the query on the parquet DataFrame
parquet_query_result = spark.sql("SELECT * FROM parquet_data WHERE price >= 350000")

# Calculate the runtime for the parquet query
parquet_runtime = time.time() - start_time

# Round the average price to two decimal places
parquet_avg_price = parquet_query_result.selectExpr("round(avg(price), 2) as avg_price")

# Show the result
parquet_avg_price.show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------+
|avg_price|
+---------+
|473796.26|
+---------+

--- 0.5975546836853027 seconds ---


In [113]:
# 14. Uncache the home_sales temporary table.
spark.table("home_sales").unpersist()

DataFrame[id: string, date: string, date_built: string, price: string, bedrooms: string, bathrooms: string, sqft_living: string, sqft_lot: string, floors: string, waterfront: string, view: string]

In [114]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')


False